In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%config Completer.use_jedi = False

In [3]:
import sys, os, pickle, time, warnings, h5py, math

import numpy as np, pandas as pd, scipy, scipy.stats as stats, tqdm, h5py
from copy import deepcopy as copy

sys.path.extend(['../utilities/', '../models/'])
import samplers, disk_cone_plcut as dcp, plotting, transformations
import disk_halo_mstogap as dh_msto
from transformations import func_inv_jac, func_labels, label_dict
import sf_utils

from multiprocessing import Pool

In [4]:
from selectionfunctions.config import config
config['data_dir'] = '/data/asfe2/Projects/testselectionfunctions/'

In [5]:
def add_log_cbar(ax, vmin=0., vmax=1., cmap='viridis',
             orientation='vertical', ticks=[0,1], tick_labels=['0','1']):

    norm = matplotlib.colors.Normalize(vmin=0, vmax=1)
    im = plt.cm.ScalarMappable(norm=norm, cmap=cmap); im.set_array([])

    tick_pos = (np.log(np.array(ticks)) - np.log(vmin))/(np.log(vmax)-np.log(vmin))
    cbar = plt.colorbar(im, cax=ax, orientation=orientation, ticks=tick_pos)

    if orientation=='vertical':
        cbar.ax.set_yticklabels(tick_labels)
    else:
        cbar.ax.set_xticklabels(tick_labels)
    #print(tick_pos, tick_labels)

    return cbar

# Load as module

In [6]:
sys.path.append('../mockmodel/')
from TracerFit import mwfit, int_idx

In [7]:
size = 'full'
file = "gaia_unwise_sdss_b80"
run_id = 12
utils_dir = '/data/asfe2/Projects/mwtrace_data/utils/'

model = {}
for cardinal in ['north', 'south']:
    
    model[cardinal] = mwfit(sf_bool=False, perr_bool=False)
    model[cardinal].load(f'/data/asfe2/Projects/mwtrace_data/gaia/gaia_{file}_{size}_{cardinal}_smaxinf_sf_perr_{run_id:03d}.h')
    model[cardinal]._generate_fid_pars(directory=utils_dir)
    
    model[cardinal+'_160'] = mwfit(sf_bool=False, perr_bool=False)
    model[cardinal+'_160'].load(f'/data/asfe2/Projects/mwtrace_data/gaia/gaia_{file}_{size}_{cardinal}_smax160_sf_perr_{run_id:03d}.h')
    model[cardinal+'_160']._generate_fid_pars(directory=utils_dir)

dict_keys(['fixed_pars', 'free_pars', 'mcmc', 'optimize', 'param_trans', 'sf_subset', 'source_id', 'true_pars'])
directory:  /data/asfe2/Projects/mwtrace_data/utils/
Got Selectionfunction pars
dict_keys(['fixed_pars', 'free_pars', 'mcmc', 'optimize', 'param_trans', 'sf_subset', 'source_id', 'true_pars'])
directory:  /data/asfe2/Projects/mwtrace_data/utils/
Got Selectionfunction pars
dict_keys(['fixed_pars', 'free_pars', 'mcmc', 'optimize', 'param_trans', 'sf_subset', 'source_id', 'true_pars'])
directory:  /data/asfe2/Projects/mwtrace_data/utils/
Got Selectionfunction pars
dict_keys(['fixed_pars', 'free_pars', 'mcmc', 'optimize', 'param_trans', 'sf_subset', 'source_id', 'true_pars'])
directory:  /data/asfe2/Projects/mwtrace_data/utils/
Got Selectionfunction pars


In [8]:
model['south'].mcmc_results['lnprob']['sf_perr_mcmc'][:,-1]

array([7179561.65065006, 7179562.09350391, 7179556.85241856,
       7179563.20508732, 7179554.73812916, 7179558.49181374,
       7179561.73732772, 7179562.8251378 , 7179562.0426747 ,
       7179563.37421071, 7179563.06903559, 7179559.7873109 ,
       7179561.44356163, 7179562.61722773, 7179563.40814815,
       7179560.83460322, 7179561.23886342, 7179561.29877097,
       7179557.736779  , 7179558.42919721, 7179562.15514131,
       7179559.87409854, 7179560.59466127, 7179560.32923724,
       7179558.94469723, 7179560.82909992, 7179559.32408991,
       7179558.77291794, 7179560.33610982, 7179561.72510044,
       7179564.64299487, 7179559.68310967, 7179561.65479328,
       7179558.35947034, 7179562.93146024, 7179562.25581121,
       7179561.62820295, 7179560.76530158, 7179560.28334728,
       7179562.28818167, 7179562.5685879 , 7179557.77814966,
       7179558.94634345, 7179558.88492112])

In [ ]:
bad_chain = model_sf.mcmc_results['lnprob']['sf_mcmc'][:,-1]<(np.max(model_sf.mcmc_results['lnprob']['sf_mcmc'][:,-1]-100))
model_sf.mcmc_results['chain']['sf_mcmc'] = model_sf.mcmc_results['chain']['sf_mcmc'][~bad_chain]

# Transformed results

In [9]:
mean = {}
err = {}

Mean mass:
- thin: 0.4127Ms
- thick: 0.3688Ms
- halo: 0.3083Ms

In [10]:
thin : 0.177, 1.083
thin : X:3.168, <M>:0.174, multiplier:0.552
thick: 0.147, 0.980
thick: X:2.786, <M>:0.168, multiplier:0.468
halo : 0.115, 0.801
halo : X:2.398, <M>:0.155, multiplier:0.372

SyntaxError: invalid syntax (<ipython-input-10-5c299fb465c2>, line 1)

In [47]:
cmpt_properties = {'thin':{'mass':0.4127,'X':1.116}, 
                   'thick':{'mass':0.3688,'X':1.094}, 
                   'halo':{'mass':0.3083,'X':1.070}}
cmpt_properties = {'thin':{'mass':0.174,'X':3.167}, 
                   'thick':{'mass':0.168,'X':2.785}, 
                   'halo':{'mass':0.155,'X':2.398}}

disk_norm = lambda h: np.tan(np.deg2rad(80))**2 / (2*np.pi*h**3)
nu_norm = dh_msto.halomodel_dist_trunc(160, 80, directory='/data/asfe2/Projects/mwtrace_data/utils/')
halo_norm = lambda n: np.tan(np.deg2rad(80))**2 / (2*np.pi) * np.exp(nu_norm(n))

cmpt_properties['thin']['rho_0'] = lambda w, h: w*cmpt_properties['thin']['mass']* disk_norm(h)/1e9 \
                                                 *cmpt_properties['thin']['X']
cmpt_properties['thick']['rho_0']= lambda w, h: w*cmpt_properties['thick']['mass']* disk_norm(h)/1e9 \
                                                 *cmpt_properties['thick']['X']
cmpt_properties['halo']['rho_0']= lambda w, h: w*cmpt_properties['halo']['mass']* halo_norm(h)/1e9 \
                                                *cmpt_properties['halo']['X']


cmpt_properties['halo']['Mfull']= lambda rho, n: -rho*1e9 * 4*np.pi/(3-n) * (160)**(3-n) / (8.27)**(-n)
rmin = 8.27
cmpt_properties['halo']['Mfull']= lambda rho, n: rho*1e9 * 4*np.pi/(3-n) * ((160)**(3-n) - (rmin)**(3-n)) * (8.27)**(n)
#rbrk=8.27; n_inner=2.75
rbrk=1; n_inner=2.
cmpt_properties['halo']['Mfull']= lambda rho, n: rho*1e9 * 4*np.pi* \
                                    ( ((160)**(3-n) - (rbrk)**(3-n)) * (8.27)**(n)/(3-n)  \
                                                    + rbrk**(3-n_inner) * (8.27)**(n_inner)/(3-n_inner) )


cmpt_properties['thin']['sd_0'] = lambda rho, h: 2*rho*h*1e3 # pc to kpc
cmpt_properties['thick']['sd_0']= lambda rho, h: 2*rho*h*1e3 # pc to kpc



def sd_halo_integral(n):
    # Performs int_-infty^infty r^-n
    integrand = lambda x: x^(-n)
    scipy.integrate.quad(integrand)
cmpt_properties['halo']['sd_0']= lambda rho, n: w*cmpt_properties['halo']['mass']* halo_norm(h)/1e9

In [48]:
cmpt_properties['halo']['Mfull'](1e-5, 3.7)/1e8

4.413655620396136

In [49]:
halo_n_grid = np.linspace(2,7,3000)
halo_sd_grid = np.zeros(len(halo_n_grid))
integrand = lambda x: x^(-n)
for ii, n in enumerate(halo_n_grid):
    integrand = lambda x: (1+x**2)**(-n/2)
    halo_sd_grid[ii] = scipy.integrate.quad(integrand, 0, 160)[0]
halo_sd_interpolated = scipy.interpolate.interp1d(halo_n_grid,halo_sd_grid)

cmpt_properties['halo']['sd_0']= lambda rho,n: 2*8.27**(n+1)*rho*halo_sd_interpolated(n)

In [50]:
parameters = [r'$\log_{10}\left(\rho^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$', 
              #r'$\log_{10}\left(M^*_\mathrm{Halo}\,\left(\mathrm{M}_\odot\right)\right)$',
              r'$\log_{10}\left(\rho_\mathrm{Tn} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$', 
              r'$\log_{10}\left(\rho_\mathrm{Tk} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$', 
              r'$\log_{10}\left(\rho_\mathrm{H} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$',
              r'$h_\mathrm{Tn} \, (\mathrm{kpc})$', r'$h_\mathrm{Tk} \, (\mathrm{kpc})$', r'$n_\mathrm{H}$',
              r'$f^\mathrm{G}_\mathrm{Tn}$', r'$f^\mathrm{G}_\mathrm{Tk}$', r'$f^\mathrm{G}_\mathrm{H}$',
              r'$\alpha_1$', r'$\alpha2$']
    
    
def transform(chain):
    
    chain_t = np.zeros((chain.shape[0], 12))

    # Total halo mass
    # chain_t[:,1] = cmpt_properties['halo']['Mfull'](chain[:,6], chain[:,7])
    # Component mass densities
    chain_t[:,1] = cmpt_properties['thin']['rho_0'](chain[:,0], chain[:,1])
    chain_t[:,2] = cmpt_properties['thick']['rho_0'](chain[:,3], chain[:,4])
    chain_t[:,3] = cmpt_properties['halo']['rho_0'](chain[:,6], chain[:,7])    
    # Total local mass density
    chain_t[:,0] = chain_t[:,[1,2,3]].sum(axis=1)
    
    # Log values
    chain_t[:,:4] = np.log10(chain_t[:,:4])
    
    # Scale height thin, height thick, power law, Dwarf fractions x 3, alpha x 2
    chain_t[:,4:] = chain[:,[1,4,7,2,5,8,9,10]]
    
    return chain_t

In [51]:
parameters = [r'$\log_{10}\left(\rho^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$', 
              #r'$\log_{10}\left(M^*_\mathrm{Halo}\,\left(\mathrm{M}_\odot\right)\right)$',
              r'$\log_{10}\left(\rho_\mathrm{Tn} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$', 
              r'$\log_{10}\left(\rho_\mathrm{Tk} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$', 
              r'$\log_{10}\left(\rho_\mathrm{H} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$',
              r'$\log_{10}\left(\Sigma^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\mathrm{pc}^2\right)\right)$', 
              #r'$\log_{10}\left(M^*_\mathrm{Halo}\,\left(\mathrm{M}_\odot\right)\right)$',
              r'$\log_{10}\left(\Sigma_\mathrm{Tn} \, \left(\mathrm{M}_\odot/\mathrm{pc}^2\right)\right)$', 
              r'$\log_{10}\left(\Sigma_\mathrm{Tk} \, \left(\mathrm{M}_\odot/\mathrm{pc}^2\right)\right)$', 
              r'$\log_{10}\left(\Sigma_\mathrm{H} \, \left(\mathrm{M}_\odot/\mathrm{pc}^2\right)\right)$',
              r'$h_\mathrm{Tn} \, (\mathrm{kpc})$', r'$h_\mathrm{Tk} \, (\mathrm{kpc})$', r'$n_\mathrm{H}$',
              r'$f^\mathrm{G}_\mathrm{Tn}$', r'$f^\mathrm{G}_\mathrm{Tk}$', r'$f^\mathrm{G}_\mathrm{H}$',
              r'$\alpha_1$', r'$\alpha2$']
    
    
parameters = [r'$\rho^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)$', 
              r'$\Sigma^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\mathrm{pc}^2\right)$', 
              #r'$M^*_\mathrm{Halo}\,\left(\mathrm{M}_\odot\right)$',
              r'$\log_{10}\left(M^*_\mathrm{Halo}\,\left(\mathrm{M}_\odot\right)\right)$',
              r'$\rho_\mathrm{Tn} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)$', 
              r'$\rho_\mathrm{Tk} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)$', 
              r'$\rho_\mathrm{H} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)$',
              r'$\rho_\mathrm{Tk}/\rho_\mathrm{Tn}$', 
              r'$\rho_\mathrm{H}/\rho_\mathrm{Tn}$', 
              r'$\Sigma_\mathrm{Tn} \, \left(\mathrm{M}_\odot/\mathrm{pc}^2\right)$', 
              r'$\Sigma_\mathrm{Tk} \, \left(\mathrm{M}_\odot/\mathrm{pc}^2\right)$', 
              r'$\Sigma_\mathrm{H} \, \left(\mathrm{M}_\odot/\mathrm{pc}^2\right)$',
              r'$h_\mathrm{Tn} \, (\mathrm{kpc})$', r'$h_\mathrm{Tk} \, (\mathrm{kpc})$', r'$n_\mathrm{H}$',
              r'$f^\mathrm{G}_\mathrm{Tn}$', r'$f^\mathrm{G}_\mathrm{Tk}$', r'$f^\mathrm{G}_\mathrm{H}$',
              r'$\alpha_1$', r'$\alpha2$']
    
def transform(chain):
    
    chain_t = np.zeros((chain.shape[0], 19))

    # Total halo mass
    # chain_t[:,1] = cmpt_properties['halo']['Mfull'](chain[:,6], chain[:,7])
    # Component mass densities
    chain_t[:,3] = cmpt_properties['thin']['rho_0'](chain[:,0], chain[:,1])
    chain_t[:,4] = cmpt_properties['thick']['rho_0'](chain[:,3], chain[:,4])
    chain_t[:,5] = cmpt_properties['halo']['rho_0'](chain[:,6], chain[:,7])    
    # Total local mass density
    chain_t[:,0] = chain_t[:,[3,4,5]].sum(axis=1)
    # Ratios
    chain_t[:,6] = chain_t[:,4]/chain_t[:,3]
    chain_t[:,7] = chain_t[:,5]/chain_t[:,3]
    
    # Component surface densities
    chain_t[:,8] = cmpt_properties['thin']['sd_0'](chain_t[:,3], chain[:,1])
    chain_t[:,9] = cmpt_properties['thick']['sd_0'](chain_t[:,4], chain[:,4])
    chain_t[:,10] = cmpt_properties['halo']['sd_0'](chain_t[:,5], chain[:,7])   
    # Total local surface density
    chain_t[:,1] = chain_t[:,[8,9,10]].sum(axis=1)
    
    chain_t[:,2] = np.log10(cmpt_properties['halo']['Mfull'](chain_t[:,5], chain[:,7]))
    
    # Log values
    # chain_t[:,:8] = np.log10(chain_t[:,:8])
    
    # Scale height thin, height thick, power law, Dwarf fractions x 3, alpha x 2
    chain_t[:,11:] = chain[:,[1,4,7,2,5,8,9,10]]
    
    return chain_t

## North and South

In [52]:
functions = [model['north_160'].fid_pars['functions'][j][par] for j in [0,1,2,'shd'] for par in model['north_160'].free_pars[j]]
labels = copy(transformations.label_set)

# Chains
chains = [model['north_160'].mcmc_results['chain']['sf_perr_mcmc'].copy(), 
          model['south_160'].mcmc_results['chain']['sf_perr_mcmc'].copy()]

# Reshape and apply functions
chains_f = []
for chain in chains: 
    nwalker, nstep, ndim = chain.shape
    chain_f = chain.copy()
    chain_f[:,:,[2,5,8]] *= -1
    chain_f = chain_f[:,2500::5].reshape(-1,ndim)
    
    for ii in range(ndim):
        chain_f[:,ii]=functions[ii](chain_f[:,ii])
    
    chains_f.append(chain_f)
    
# Transform to parameters of interest
chains_t = []
for chain_f in chains_f:
    chains_t.append(transform(chain_f.copy()))

In [53]:
mean['north'] = np.mean(chains_t[0], axis=0)
mean['south'] = np.mean(chains_t[1], axis=0)

err['north'] = np.std(chains_t[0], axis=0)
err['south'] = np.std(chains_t[1], axis=0)
err['combined'] = np.sqrt((err['north']**2 + err['north']**2)/2)

In [54]:
err['north']

array([2.92114867e-04, 9.03775077e-02, 1.42119662e-02, 2.61872396e-04,
       1.29215236e-04, 3.11921918e-07, 4.72170466e-03, 1.39253622e-05,
       1.47345272e-01, 1.28903843e-01, 4.20810880e-02, 2.50025517e-03,
       6.65993191e-03, 2.28382482e-02, 4.31849890e-04, 1.60517604e-04,
       9.54194431e-05, 4.68492878e-04, 2.26228221e-03])

In [55]:
mean['south']

array([ 4.05178707e-02,  2.48262920e+01,  8.74313295e+00,  3.55786527e-02,
        4.92408168e-03,  1.51363243e-05,  1.38407453e-01,  4.25472942e-04,
        1.78205924e+01,  6.71992218e+00,  2.85777480e-01,  2.50450029e-01,
        6.82457705e-01,  3.38192467e+00,  1.36336357e-02,  5.87248846e-03,
        4.66569544e-03, -1.07461604e-01, -2.78534658e-01])

In [56]:
mean['north']

array([ 3.27794013e-02,  2.20203644e+01,  8.97493439e+00,  2.85632708e-02,
        4.19483452e-03,  2.12959196e-05,  1.46873466e-01,  7.45651414e-04,
        1.53610216e+01,  5.92129136e+00,  7.38051393e-01,  2.68905202e-01,
        7.05982688e-01,  3.70489859e+00,  1.24808817e-02,  5.21255724e-03,
        5.90518063e-03, -1.12342322e-01, -2.70467717e-01])

### Estimate mean and systematic error

In [57]:
def likelihood_sys(params, mean, err, grad=False):
    mu, sys = params.reshape(2,-1)
    sys = np.exp(sys)
    lnL = -(mean - mu[None,:])**2/(2*(err**2 + sys[None,:]**2)) - 0.5*np.log(err**2 + sys[None,:]**2)
    
    if not grad: return -np.sum(lnL)
    
    grad = np.zeros(len(params))
    grad[:len(mu)] = np.sum( (mean - mu[None,:])/(err**2 + sys[None,:]**2), axis=0 )
    grad[len(mu):] = np.sum( sys[None,:]*(mean - mu[None,:])**2/(err**2 + sys[None,:]**2)**2 \
                             -sys[None,:]/(err**2 + sys[None,:]**2) , axis=0 )*sys
    
    return -np.sum(lnL), -grad

In [58]:
p0 = np.random.rand(len(mean['north'])*2)

mu = np.vstack((mean['north'], mean['south']))
std = np.vstack((err['north'], err['south']))

foo = lambda x: likelihood_sys(x, mu, std, grad=False)
likelihood_sys(p0, mu, std, grad=True), scipy.optimize.approx_fprime(p0, foo, 1e-8)

((403.03390636431,
  array([ 9.30450952e-01, -6.47750939e+00, -4.41819214e+00,  4.27102467e-01,
          8.92697332e-01,  3.83774922e-01, -1.55584515e-01,  3.98287150e-01,
         -3.15928732e+01, -9.49572447e+00,  4.86483855e-02, -3.52974032e-02,
         -8.71669319e-01, -2.76785440e+00,  1.85459161e-02,  2.56923451e-01,
          7.65107296e-02,  2.49179826e-01,  5.27344437e-01,  1.41251655e+00,
         -1.47497664e+02, -3.27649941e+01,  1.88697025e+00,  1.39073149e+00,
          1.68599853e+00,  1.98161939e+00,  1.82604564e+00, -5.06230372e+02,
         -5.28676972e+01,  1.97194135e+00,  1.99542130e+00,  1.41514742e+00,
         -6.00248199e+00,  1.99949832e+00,  1.84630234e+00,  1.98147190e+00,
          1.89874392e+00,  1.57172585e+00])),
 array([ 9.30452866e-01, -6.47750653e+00, -4.41819450e+00,  4.27104397e-01,
         8.92697467e-01,  3.83772656e-01, -1.55586122e-01,  3.98290467e-01,
        -3.15928730e+01, -9.49572149e+00,  4.86522820e-02, -3.52940788e-02,
        -8.716

In [59]:
foo = lambda x: likelihood_sys(x, mu, std, grad=True)
result = scipy.optimize.minimize(foo, p0, jac=True, method='BFGS')
result['x']

array([ 3.66434916e-02,  2.34226633e+01,  8.85847815e+00,  3.20660340e-02,
        4.56508793e-03,  1.82070580e-05,  1.41114399e-01,  5.85119656e-04,
        1.65923502e+01,  6.32668097e+00,  5.08440723e-01,  2.59542336e-01,
        6.93298260e-01,  3.54302513e+00,  1.30627362e-02,  5.54591721e-03,
        5.28457433e-03, -1.09894715e-01, -2.74529964e-01, -5.55822650e+00,
        3.36269697e-01, -2.16018089e+00, -5.65629978e+00, -7.97512939e+00,
       -1.26943218e+01, -1.93624062e+01, -8.74218988e+00,  2.00245344e-01,
       -9.64736673e-01, -1.49646423e+00, -4.71594999e+00, -4.58590035e+00,
       -1.83221601e+00, -7.86018706e+00, -8.14491798e+00, -7.39748965e+00,
       -6.03285076e+00, -5.69685184e+00])

In [60]:
bfgs_result = np.zeros(2*len(parameters))

for ii in range(len(parameters)):
    p0 = np.random.normal(0,1,2) 
    p0[0] += np.mean(mu[:,ii:ii+1])
    p0[1] *= np.mean(std[:,ii:ii+1])
    if ii==2: print(mu[:,ii:ii+1], std[:,ii:ii+1])
    foo = lambda x: likelihood_sys(x, mu[:,ii:ii+1], std[:,ii:ii+1], grad=True)
    result = scipy.optimize.minimize(foo, p0, jac=True, method='BFGS')
    result['x']

    bfgs_result[[ii, len(parameters)+ii]] = result['x']

[[8.97493439]
 [8.74313295]] [[0.01421197]
 [0.00855635]]


In [62]:
mean['north'], mean['south'], bfgs_result[:len(mean['north'])], bfgs_result[len(mean['north']):]

(array([ 3.27794013e-02,  2.20203644e+01,  8.97493439e+00,  2.85632708e-02,
         4.19483452e-03,  2.12959196e-05,  1.46873466e-01,  7.45651414e-04,
         1.53610216e+01,  5.92129136e+00,  7.38051393e-01,  2.68905202e-01,
         7.05982688e-01,  3.70489859e+00,  1.24808817e-02,  5.21255724e-03,
         5.90518063e-03, -1.12342322e-01, -2.70467717e-01]),
 array([ 4.05178707e-02,  2.48262920e+01,  8.74313295e+00,  3.55786527e-02,
         4.92408168e-03,  1.51363243e-05,  1.38407453e-01,  4.25472942e-04,
         1.78205924e+01,  6.71992218e+00,  2.85777480e-01,  2.50450029e-01,
         6.82457705e-01,  3.38192467e+00,  1.36336357e-02,  5.87248846e-03,
         4.66569544e-03, -1.07461604e-01, -2.78534658e-01]),
 array([ 3.66434916e-02,  2.34226672e+01,  8.85847819e+00,  3.20660340e-02,
         4.56508793e-03,  1.82070580e-05,  1.41114398e-01,  5.85119656e-04,
         1.65923502e+01,  6.32668065e+00,  5.08440818e-01,  2.59542336e-01,
         6.93298260e-01,  3.54302515e+00, 

In [63]:
result['x']

array([-0.27452996, -5.69685192])

In [64]:
mean['combined'] = bfgs_result[:len(mean['north'])].copy()
err['sys_ns'] = np.exp(bfgs_result[len(mean['north']):].copy())

In [65]:
mean['combined']

array([ 3.66434916e-02,  2.34226672e+01,  8.85847819e+00,  3.20660340e-02,
        4.56508793e-03,  1.82070580e-05,  1.41114398e-01,  5.85119656e-04,
        1.65923502e+01,  6.32668065e+00,  5.08440818e-01,  2.59542336e-01,
        6.93298260e-01,  3.54302515e+00,  1.30627362e-02,  5.54591721e-03,
        5.28457433e-03, -1.09894715e-01, -2.74529964e-01])

In [66]:
err['sys_ns']

array([3.85560824e-03, 1.39971431e+00, 1.15304322e-01, 3.49542688e-03,
       3.43910402e-04, 3.06849981e-06, 8.65192016e-07, 1.59703771e-04,
       1.22170250e+00, 3.81083771e-01, 2.23920409e-01, 8.95135830e-03,
       1.01945670e-02, 1.60058079e-01, 3.85801696e-04, 2.90206453e-04,
       6.12789150e-04, 2.39864626e-03, 3.35651541e-03])

# Systematic Tests

In [67]:
sys_labels = {'good':'Good', 'z0':r'$z_\odot$', 'dust':r'$A_V$', 'gerr':r'$\sigma_G$',
             'poff':r'$\Delta\varpi$', 'mto':r'$M_\mathrm{TO}^\mathrm{Tn}$', 'R':r'$R$'}
systematics = ['good', 'z0', 'dust', 'gerr', 'poff', 'mto', 'R']

run_id=7; run_id_mock=28
size = 1000000
file = "sample4"

sys_model = {}

sys_model['good'] = mwfit(sf_bool=False, perr_bool=False)
sys_model['good'].load(f'/data/asfe2/Projects/mwtrace_data/mockmodel/mock_{file}_{size}_sf_perr_{run_id_mock:03d}.h')

sys_model['z0'] = mwfit(sf_bool=False, perr_bool=False)
sys_model['z0'].load(f'/data/asfe2/Projects/mwtrace_data/mockmodel/mock_sys_{file}_{size}_z0_{run_id:03d}.h')

sys_model['dust'] = mwfit(sf_bool=False, perr_bool=False)
sys_model['dust'].load(f'/data/asfe2/Projects/mwtrace_data/mockmodel/mock_sys_{file}_{size}_dust_{run_id:03d}.h')

sys_model['gerr'] = mwfit(sf_bool=False, perr_bool=False)
sys_model['gerr'].load(f'/data/asfe2/Projects/mwtrace_data/mockmodel/mock_sys_{file}_{size}_gerr_{run_id:03d}.h')

sys_model['poff'] = mwfit(sf_bool=False, perr_bool=False)
sys_model['poff'].load(f'/data/asfe2/Projects/mwtrace_data/mockmodel/mock_sys_{file}_{size}_poffset_{run_id:03d}.h')

sys_model['mto'] = mwfit(sf_bool=False, perr_bool=False)
sys_model['mto'].load(f'/data/asfe2/Projects/mwtrace_data/mockmodel/mock_sample5_{size}_sf_perr_{run_id_mock:03d}.h')

sys_model['R'] = mwfit(sf_bool=False, perr_bool=False)
sys_model['R'].load(f'/data/asfe2/Projects/mwtrace_data/mockmodel/mock_sample7_{size}_sf_perr_{run_id_mock:03d}.h')

for sys in systematics:
    sys_model[sys]._generate_fid_pars(directory=utils_dir)

dict_keys(['fixed_pars', 'free_pars', 'mcmc', 'optimize', 'param_trans', 'sf_subset', 'source_id', 'true_pars'])
dict_keys(['fixed_pars', 'free_pars', 'mcmc', 'optimize', 'param_trans', 'sf_subset', 'source_id', 'true_pars'])
dict_keys(['fixed_pars', 'free_pars', 'mcmc', 'optimize', 'param_trans', 'sf_subset', 'source_id', 'true_pars'])
dict_keys(['fixed_pars', 'free_pars', 'mcmc', 'optimize', 'param_trans', 'sf_subset', 'source_id', 'true_pars'])
dict_keys(['fixed_pars', 'free_pars', 'mcmc', 'optimize', 'param_trans', 'sf_subset', 'source_id', 'true_pars'])
dict_keys(['fixed_pars', 'free_pars', 'mcmc', 'optimize', 'param_trans', 'sf_subset', 'source_id', 'true_pars'])
dict_keys(['fixed_pars', 'free_pars', 'mcmc', 'optimize', 'param_trans', 'sf_subset', 'source_id', 'true_pars'])
directory:  /data/asfe2/Projects/mwtrace_data/utils/
Got Selectionfunction pars
directory:  /data/asfe2/Projects/mwtrace_data/utils/
Got Selectionfunction pars
directory:  /data/asfe2/Projects/mwtrace_data/uti

In [68]:
labels = copy(transformations.label_set)

# Chains
sys_chains = {sys:sys_model[sys].mcmc_results['chain']['sf_perr_mcmc'].copy() for sys in systematics}

# Reshape and apply functions
sys_chains_f = {}
for i, sys in enumerate(systematics):
    sys_chain = sys_chains[sys].copy()
    sys_chain[:,:,[2,5,8]] *= -1
    functions = [sys_model[sys].fid_pars['functions'][j][par] for j in [0,1,2,'shd']\
                 for par in sys_model[sys].free_pars[j]]
    
    nwalker, nstep, ndim = sys_chain.shape
    sys_chain_f = sys_chain.copy()
    sys_chain_f = sys_chain_f[:,2500::5].reshape(-1,ndim)
    
    for ii in range(ndim):
        sys_chain_f[:,ii]=functions[ii](sys_chain_f[:,ii])
    
    sys_chains_f[sys] = sys_chain_f
    
# Transform to parameters of interest
sys_chains_t = {}
for i, sys in enumerate(systematics):
    sys_chains_t[sys] = transform(sys_chains_f[sys].copy())

### True parameters

In [69]:
# Load Sample
file_truths = {}
files = ['sample4', 'sample5', 'sample7']
for file in files:
    file_truths[file]={};
    
    filename = f'/data/asfe2/Projects/mwtrace_data/mockmodel/{file}.h'
    with h5py.File(filename, 'r') as hf:
        for key in hf['true_pars'].keys():
            if not key in np.arange(3).astype(str):
                file_truths[file][int_idx(key)]=hf['true_pars'][key][...]
            else:
                file_truths[file][int_idx(key)]={}
                for par in hf['true_pars'][key].keys():
                    file_truths[file][int_idx(key)][par]=hf['true_pars'][key][par][...]
    for j in range(3): file_truths[file][j]['w']*=size
sys_files = dict({sys:'sample4' for sys in ['good', 'z0', 'dust', 'gerr', 'poff']}, \
                 **{'mto':'sample5', 'R':'sample7'})

In [70]:
sys_true = {}
sys_true_t = {}
for sys in systematics:
    sys_true[sys] = sys_model[sys].get_true_params(file_truths[sys_files[sys]]).copy()
    sys_true[sys][[2,5,8]] = 1-sys_true[sys][[2,5,8]]
    
    sys_true_t[sys] = transform(sys_true[sys].copy()[None,:])[0]

In [71]:
sys_true['good'], np.mean(sys_chains_f['good'], axis=0), np.std(sys_chains_f['good'], axis=0)

(array([ 1.20e+04,  3.00e-01,  4.50e-03,  4.30e+04,  9.00e-01,  5.40e-03,
         9.45e+05,  3.74e+00,  3.50e-03, -1.10e-01, -2.50e-01]),
 array([ 9.61362380e+03,  2.81277373e-01,  3.82626958e-03,  4.19288883e+04,
         8.15565529e-01,  5.85413725e-03,  8.84161609e+05,  3.81282628e+00,
         3.48211147e-03, -1.09847145e-01, -2.52067559e-01]),
 array([1.36915733e+03, 1.50653778e-02, 1.38662989e-03, 1.92295979e+03,
        4.95320306e-02, 6.73698597e-04, 5.01805983e+04, 6.77330109e-02,
        1.47782301e-04, 2.04636681e-03, 8.59353409e-03]))

In [72]:
sys_true_t['good'], np.mean(sys_chains_t['good'], axis=0), np.std(sys_chains_t['good'], axis=0)

(array([ 1.40155556e-03,  1.24959526e+00,  8.47713046e+00,  1.25371085e-03,
         1.41272620e-04,  6.57209724e-06,  1.12683574e-01,  5.24211565e-03,
         7.52226508e-01,  2.54290715e-01,  2.43078041e-01,  3.00000000e-01,
         9.00000000e-01,  3.74000000e+00,  4.50000000e-03,  5.40000000e-03,
         3.50000000e-03, -1.10000000e-01, -2.50000000e-01]),
 array([ 1.40778221e-03,  1.27152994e+00,  8.51206688e+00,  1.21278858e-03,
         1.88341185e-04,  6.65244610e-06,  1.55568883e-01,  5.49864703e-03,
         6.81392621e-01,  3.04360579e-01,  2.85776737e-01,  2.81277373e-01,
         8.15565529e-01,  3.81282628e+00,  3.82626958e-03,  5.85413725e-03,
         3.48211147e-03, -1.09847145e-01, -2.52067559e-01]),
 array([6.71579022e-05, 4.49123284e-02, 3.72663938e-02, 5.78283943e-05,
        3.00372781e-05, 1.68804058e-07, 2.55606331e-02, 3.14461601e-04,
        3.45998788e-02, 3.13314715e-02, 4.34454592e-02, 1.50653778e-02,
        4.95320306e-02, 6.77330109e-02, 1.38662989e-03

In [73]:
sys_true_t['good']

array([ 1.40155556e-03,  1.24959526e+00,  8.47713046e+00,  1.25371085e-03,
        1.41272620e-04,  6.57209724e-06,  1.12683574e-01,  5.24211565e-03,
        7.52226508e-01,  2.54290715e-01,  2.43078041e-01,  3.00000000e-01,
        9.00000000e-01,  3.74000000e+00,  4.50000000e-03,  5.40000000e-03,
        3.50000000e-03, -1.10000000e-01, -2.50000000e-01])

In [74]:
mean['combined']

array([ 3.66434916e-02,  2.34226672e+01,  8.85847819e+00,  3.20660340e-02,
        4.56508793e-03,  1.82070580e-05,  1.41114398e-01,  5.85119656e-04,
        1.65923502e+01,  6.32668065e+00,  5.08440818e-01,  2.59542336e-01,
        6.93298260e-01,  3.54302515e+00,  1.30627362e-02,  5.54591721e-03,
        5.28457433e-03, -1.09894715e-01, -2.74529964e-01])

In [75]:
np.mean(sys_chains_t['R'][:,7]), np.std(sys_chains_t['R'][:,7])

(0.006230185991252767, 0.00035952945648988564)

### Likelihood function

In [76]:
def likelihood_sys(params, x, mu, err, grad=False):
    
    if np.sum(params<-20)+np.sum(params>20)>0:
        if not grad: return 1e20
        else: return 1e20, np.zeros(len(params))
    
    sys = np.exp(params)
    lnL = -(x - mu[None,:])**2/(2*(err**2 + sys[None,:]**2)) - 0.5*np.log(err[None,:]**2 + sys[None,:]**2)
    
    if not grad: return -np.sum(lnL)
    
    grad = np.sum( sys[None,:]*(x - mu[None,:])**2/(err[None,:]**2 + sys[None,:]**2)**2 \
                             -sys[None,:]/(err[None,:]**2 + sys[None,:]**2) , axis=0 )*sys
    
    return -np.sum(lnL), -grad

In [77]:
sys='good'
bounds = np.vstack((np.zeros(len(parameters))-10, np.zeros(len(parameters))+10)).T
p0 = np.random.rand(bounds.shape[0])
statistical_error = np.std(sys_chains_t[sys], axis=0)
for ii in range(len(parameters)):
    foo = lambda x: likelihood_sys(x, sys_chains_t[sys][:,ii:ii+1],
                                      sys_true_t[sys][ii:ii+1],  statistical_error[ii:ii+1], grad=True)
    result = scipy.optimize.minimize(foo, p0[ii:ii+1], jac=True, method='Newton-CG')#, bounds=bounds[ii:ii+1])
    print(result['success'], result['fun'], result['x'][0])

True -200292.05146180867 -11.986672800720012
True -54913.82543392858 -3.819686659634879
True -54435.246608548114 -3.3542253116577427
True -199210.92283480914 -10.103836181799318
True -204447.91584964734 -9.963905187029683
False 9190.56797622097 0.41775308982821224
True -54939.17868566794 -3.1492259627783294
True -160810.9131187799 -8.26825957046454
True -44889.408868387196 -2.6474177705570643
True -51240.63733830993 -2.994335972096109
True -50561.92211711951 -3.153586910672722
True -71024.77809964705 -3.9780224827183166
True -40125.86732219515 -2.471779535297249
True -39776.95487201917 -2.6196784303973772
True -131447.9811397894 -7.3026804922858055
True -145539.44553081508 -7.697111099214818
False -182874.94127896754 -10.931350768487174
True -125155.96247328754 -8.786023179542294
True -93029.39790294039 -6.181386450598742


In [248]:
err['sys_tests'] = np.zeros((len(systematics), len(parameters)))
bounds = np.vstack((np.zeros(len(parameters))-15, np.zeros(len(parameters))+10)).T
for ii, sys  in enumerate(systematics):
    
    print(sys)
    p0 = np.random.rand(bounds.shape[0])
    statistical_error = np.std(sys_chains_t[sys], axis=0)
    foo = lambda x: likelihood_sys(x, sys_chains_t[sys],
                                      sys_true_t[sys],  statistical_error, grad=True)
    result = scipy.optimize.minimize(foo, p0, jac=True, method='BFGS', bounds=bounds)
    print(result)
    result['x']
    
    err['sys_tests'][ii] = np.exp(result['x'].copy())

good


/home/andy/py36/lib/python3.6/site-packages/scipy/optimize/_minimize.py:534: RuntimeWarning: Method BFGS cannot handle constraints nor bounds.
  RuntimeWarning)


      fun: -1112694.7794789975
 hess_inv: array([[ 1.88165299e-01, -3.01000891e-05,  4.30803331e-05,
         8.57312314e-06,  1.21016937e-04, -6.55450796e-06,
         1.13993796e-05,  2.76510952e-05, -1.47956799e-01,
         1.19157534e-01,  1.39097617e-01, -2.78888727e-01,
         2.08598870e-04],
       [-3.01000891e-05,  7.10894164e-05, -6.21591197e-06,
         4.09697858e-06, -7.67943106e-06, -5.62452041e-07,
         2.45488876e-06, -1.12428953e-06,  1.26131234e-06,
        -1.13217960e-05, -1.22611221e-05,  1.54011455e-05,
        -3.65642065e-05],
       [ 4.30803331e-05, -6.21591197e-06,  1.73050215e-04,
         2.02467039e-06, -1.39488387e-05, -1.00977957e-06,
         1.59367610e-06, -2.18515998e-06, -1.68219147e-04,
         1.24783173e-04,  1.46655793e-04, -3.16616038e-04,
         1.62952289e-05],
       [ 8.57312314e-06,  4.09697858e-06,  2.02467039e-06,
         4.28747116e-05,  3.98496920e-06,  1.81885417e-06,
        -2.21614429e-07,  2.12909067e-06, -6.21757048e

/home/andy/py36/lib/python3.6/site-packages/scipy/optimize/_minimize.py:534: RuntimeWarning: Method BFGS cannot handle constraints nor bounds.
  RuntimeWarning)


      fun: -1075055.1594576342
 hess_inv: array([[ 3.49510559e-05,  1.62776089e-06, -7.87377776e-07,
         3.74320975e-07, -5.04709819e-06,  7.77368802e-07,
        -9.47231653e-07, -1.34722364e-06,  1.47120013e-05,
         1.58802025e-05,  3.34865072e-05, -5.58626768e-05,
        -9.76121215e-06],
       [ 1.62776089e-06,  2.18207444e-04, -9.46790980e-07,
         8.80472050e-06, -2.13741101e-05,  3.26256797e-05,
         1.22076169e-05,  2.17184365e-06,  3.91043802e-05,
         4.82248811e-05,  1.85682639e-04, -5.11822877e-04,
        -7.05520559e-06],
       [-7.87377776e-07, -9.46790980e-07,  2.45021743e-05,
        -4.47224709e-07, -1.56834374e-06,  6.21729368e-06,
        -4.21193718e-07,  8.79649018e-08,  1.62815041e-05,
         1.78597429e-05,  4.16458453e-05, -7.91279133e-05,
         9.89082162e-07],
       [ 3.74320975e-07,  8.80472050e-06, -4.47224709e-07,
         7.31876790e-04, -1.09109689e-06, -5.18647071e-05,
        -8.59289238e-06, -6.67176864e-08,  1.17795081e

...

      fun: -1034659.6370998656
 hess_inv: array([[ 7.63976699e-03, -4.68733755e-08,  2.26483851e-06,
         5.00781679e-07, -7.78091725e-03, -2.74711699e-06,
         2.22978644e-06, -9.67803827e-07,  7.15731428e-07,
        -2.67506421e-06,  3.17399337e-07,  2.28641583e-06,
         3.79615240e-06],
       [-4.68733755e-08,  5.89290976e-05, -8.47807623e-08,
        -4.26495648e-07, -5.90696938e-05,  2.93422496e-07,
        -1.02384485e-07, -5.10808326e-08,  1.54904569e-07,
        -1.37320964e-07, -1.87532490e-07,  1.37800306e-07,
        -4.38707506e-08],
       [ 2.26483851e-06, -8.47807623e-08,  6.81112397e-05,
        -1.24908659e-07, -8.57436567e-05,  7.55303020e-07,
         8.50128733e-07,  3.22258244e-06,  7.04307389e-07,
        -1.22281476e-06, -1.24287266e-06,  4.85206035e-07,
        -1.81811692e-07],
       [ 5.00781679e-07, -4.26495648e-07, -1.24908659e-07,
         4.46922506e-05, -4.71107966e-05, -4.85920419e-07,
         1.33737454e-06, -3.08447323e-07,  1.40029902e

/home/andy/py36/lib/python3.6/site-packages/scipy/optimize/_minimize.py:534: RuntimeWarning: Method BFGS cannot handle constraints nor bounds.
  RuntimeWarning)


      fun: -1134909.6464215256
 hess_inv: array([[ 5.99460197e-04, -5.41921699e-06, -3.72177432e-06,
        -5.72666551e-04, -3.36859896e-05,  1.85107151e-05,
         6.65692258e-05, -7.01798116e-07, -3.10425780e-08,
        -5.18781110e-04,  8.16488382e-04,  8.07287878e-09,
         7.94167769e-05],
       [-5.41921699e-06,  9.56011575e-05,  2.18046227e-05,
        -1.27417388e-05,  5.62856544e-06,  1.68817654e-05,
        -9.87017016e-06,  4.53187081e-06,  1.10744708e-07,
        -3.55418272e-04,  3.73597894e-04, -1.41274476e-09,
        -3.53522249e-05],
       [-3.72177432e-06,  2.18046227e-05,  5.03174277e-04,
        -1.38793301e-04, -2.14904935e-05, -4.38436660e-06,
         1.63209187e-05, -2.00633816e-06, -6.44711646e-08,
        -1.49919711e-03,  1.83487522e-03, -3.05542225e-10,
        -7.43306838e-06],
       [-5.72666551e-04, -1.27417388e-05, -1.38793301e-04,
         1.42422850e+00,  4.73319908e-05, -2.02933150e-04,
         1.03745038e+00, -1.03411007e-04, -1.55388315e

/home/andy/py36/lib/python3.6/site-packages/scipy/optimize/_minimize.py:534: RuntimeWarning: Method BFGS cannot handle constraints nor bounds.
  RuntimeWarning)


      fun: -984796.980664638
 hess_inv: array([[ 1.19222433e-04, -1.82438825e-04,  2.59315467e-04,
         5.92122106e-07,  1.22798289e-04, -6.16242979e-02,
        -3.78849690e-04,  2.17334593e-04,  6.22102504e-01,
        -1.72843071e+00,  9.51617659e-05,  1.16596715e+00,
        -7.94176681e-05],
       [-1.82438825e-04,  4.39299403e-04, -5.79882616e-04,
        -1.71000797e-06, -2.75655283e-04,  1.37614480e-01,
         8.49039877e-04, -4.81175959e-04, -1.39024674e+00,
         3.86284824e+00, -2.12265748e-04, -2.60588672e+00,
         1.79190968e-04],
       [ 2.59315467e-04, -5.79882616e-04,  8.74036223e-04,
         2.25845924e-06,  3.91272802e-04, -1.95901976e-01,
        -1.20818015e-03,  6.86191394e-04,  1.97898634e+00,
        -5.49862397e+00,  3.02605732e-04,  3.70934202e+00,
        -2.54870225e-04],
       [ 5.92122106e-07, -1.71000797e-06,  2.25845924e-06,
         4.61443259e-05,  1.36964013e-06, -5.93346399e-04,
        -3.11881332e-06,  1.77929412e-06,  5.35634193e-0

In [78]:
err['sys_tests_1b1'] = np.zeros((len(systematics), len(parameters)))
bounds = np.vstack((np.zeros(len(parameters))-20, np.zeros(len(parameters))+10)).T
for isys, sys  in enumerate(systematics):
    
    print(sys)
    statistical_error = np.std(sys_chains_t[sys], axis=0)
    for ii in range(len(parameters)):
        x = 0; lnL = 1e20
        for itrial in range(10):
            p0 = np.random.normal(0,2,1)
            foo = lambda x: likelihood_sys(x, sys_chains_t[sys][:,ii:ii+1],
                                              sys_true_t[sys][ii:ii+1],  statistical_error[ii:ii+1], grad=True)
            result = scipy.optimize.minimize(foo, p0, jac=True, method='BFGS')#, bounds=bounds[ii:ii+1])
            if result['fun']<lnL:
                x = result['x']
                lnL =result['fun']
            #print(result['success'], result['fun'], result['x'][0])
        print(x, lnL)
        err['sys_tests_1b1'][isys,ii] = np.exp(x.copy())

good
[-11.98667277] -200292.0514618087
[-3.81968666] -54913.82543392858
[-3.35422531] -54435.2466085481
[-10.10383618] -199210.92283480917
[-9.96390519] -204447.91584964737
[-16.33688804] -329833.2017809388
[-3.14922596] -54939.17868566794
[-8.26825957] -160810.9131187799
[-2.64741777] -44889.408868387196
[-2.99433597] -51240.63733830993
[-3.15358691] -50561.922117119524
[-3.97802248] -71024.77809964707
[-2.47177954] -40125.86732219515
[-2.61967843] -39776.95487201917
[-7.30268049] -131447.9811397894
[-7.6971111] -145539.4455308151
[-10.93135099] -182874.94127896754
[-8.78602345] -125155.96247328754
[-6.18138645] -93029.39790294039
z0
[-8.94440388] -183638.400841571
[-2.91646567] -46975.752526703574
[-3.59343934] -55055.38683609719
[-8.80968221] -181628.14994593587
[-10.88532457] -211968.38671060101
[-16.2882503] -328362.5828669945
[-3.42501076] -56598.28053037231
[-7.13185085] -143929.45016168617
[-2.29688538] -37886.53498298509
[-4.32476248] -59984.232540439196
[-3.40489639] -51583.6

good
[-11.9406101] -206299.20057075378
[-3.82426901] -58039.96144047626
[5.8604035] 393914.55873171834
[-10.39295531] -205571.54369508408
[-10.20186255] -209682.97779384884
[-16.52387488] -333946.912230853
[-3.09806419] -53813.61976959453
[-8.16612728] -158564.00270841917
[-2.9365369] -51250.02972866208
[-3.23229333] -56475.69928251144
[-3.34057375] -54675.632567033725
[-3.97802248] -71024.77809964705
[-2.47177954] -40125.86732219515
[-2.61967843] -39776.95487201917
[-7.30268049] -131447.9811397894
[-7.6971111] -145539.44553081508
[-10.931351] -182874.94127896754
[-8.78602344] -125155.96247328754
[-6.18138645] -93029.39790294039
z0
[-9.24115813] -190068.99258953065
[-3.2876812] -53037.79503043957
[2.28621709] 391482.28459645296
[-9.09880134] -187988.7708062108
[-11.12328192] -217203.4486548025
[-16.47523713] -332476.29331690865
[-3.373849] -55472.721614298905
[-7.02971856] -141682.53975132547
[-2.58600451] -44247.15584325997
[-4.56271984] -65219.29448464069
[-3.59188323] -55697.3621994

good
[-11.9406101] -206299.2005707538
[-10.39295531] -205571.54369508408
[-10.20186255] -209682.97779384884
[-16.52387488] -333946.912230853
[-3.82426901] -58039.96144047627
[-2.9365369] -51250.02972866208
[-3.23229333] -56475.69928251144
[-3.34057375] -54675.632567033725
[-3.97802248] -71024.77809964705
[-2.47177954] -40125.86732219515
[-2.61967843] -39776.954872019174
[-7.30268049] -131447.9811397894
[-7.6971111] -145539.44553081508
[-10.93135091] -182874.94127896754
[-8.78602104] -125155.96247328754
[-6.18138643] -93029.3979029404
z0
[-9.24115813] -190068.99258953068
[-9.09880134] -187988.7708062108
[-11.12328192] -217203.44865480252
[-16.47523714] -332476.29331690865
[-3.2876812] -53037.79503043957
[-2.58600451] -44247.15584325997
[-4.56271984] -65219.29448464069
[-3.59188322] -55697.36219948119
[-5.36228065] -76501.51832743955
[-3.00858808] -42924.57477287323
[-2.93769287] -40886.38728670607
[-6.6812723] -127805.72667164697
[-7.40285309] -141630.89922759455
[-11.22096018] -180513.

good
[-6.12930874] -74157.87425893501
[-1.81686278] -22381.445522832913
[-4.20607743] -69731.0952549075
[-2.12599901] -32521.274559128484
[-5.27105315] -86072.4649822306
[-3.97802248] -71024.77809964705
[-2.47177954] -40125.86732219515
[-2.61967843] -39776.95487201917
[-7.30268049] -131447.9811397894
[-7.69711109] -145539.44553081508
[-10.93135084] -182874.94127896754
[-8.78602345] -125155.96247328754
[-6.18138645] -93029.39790294039
z0
[-3.16207631] -56196.379207387305
[-2.15655506] -23536.749865093905
[-2.89119463] -51290.531790091176
[-3.12895642] -38656.239076715545
[-5.22528872] -84643.43994429609
[-5.36228065] -76501.51832743955
[-3.00858806] -42924.57477287323
[-2.93769287] -40886.38728670607
[-6.6812723] -127805.72667164699
[-7.40285309] -141630.89922759455
[-11.2209644] -180513.2588724563
[-8.77683113] -123797.57362653734
[-6.05368231] -91190.04156349238
dust
[-4.86082767] -71991.97813241198
[-1.12359457] -11399.25545456542
[-4.13506512] -68254.26615009522
[-1.82850828] -26538

good
True -2.6181369825878006e-07 -15.0
True -5.761515746565968e-07 -15.0
True -1.2578949310055241e-05 -14.99999949811741
True -6.140274446441985e-06 -15.0
True 0.000522112574491778 -5.27398063094288
True -4.809676036458794e-06 -3.9793591901778242
True -1.924348996765011e-05 -2.4730759993290006
True -2.5811011280662515e-06 -15.0
True 0.0005912354216312218 -7.305356873703357
True 0.00011433160571569861 -7.697782567983734
True -0.002069792951580892 -10.916754608979765
True -1.3440430436906396e-05 -14.999999349387453
True -8.09737379356473e-06 -15.0
z0
False -9.811095633680633e-05 -15.0
True -1.535231594185493e-07 -15.0
True 1.92312200443357e-06 -2.891421520548829
True -2.405226947443924e-07 -15.0
False -1.547743906708186e-05 -15.0
True -2.0833041838059227e-06 -15.0
True -1.0012972014347095e-06 -15.0
True -7.65136519991333e-07 -15.0
True -0.0002745105757611409 -6.681429357305338
True -0.03136164447612978 -7.4023653092484425
True -0.020882457090612587 -11.23147162766709
True -1.11105524003

In [59]:
err['sys_rms'] = np.sqrt(np.sum(err['sys_tests']**2, axis=0))
err['sys_max'] = np.max(err['sys_tests'], axis=0)

KeyError: 'sys_tests'

In [405]:
sys_dominant = [systematics[i] for i in np.argmax(err['sys_tests'], axis=0)]
print(sys_dominant)

['z0', 'R', 'z0', 'dust', 'R', 'gerr', 'R', 'R', 'mto', 'poff', 'poff', 'poff', 'poff']


In [406]:
print([systematics[i] for i in np.argmin(err['sys_tests'], axis=0)])

['good', 'gerr', 'mto', 'mto', 'poff', 'R', 'mto', 'gerr', 'z0', 'mto', 'mto', 'z0', 'dust']


# Results Table

In [60]:
columns = [r'$\mu^\mathrm{North}$', r'$\sigma^\mathrm{North}$', 
           r'$\mu^\mathrm{South}$', r'$\sigma^\mathrm{South}$',
           r'$\mu$', r'$\sigma$']
mean_cols = ['north', 'south', 'combined']
mean_labels = [r'$\mu^\mathrm{North}$', r'$\mu^\mathrm{South}$', r'$\mu$']

err_cols = ['sys_ns', 'sys_max']
err_labels = [r'$\sigma_\mathrm{sys}^{\mathrm{N}/\mathrm{S}}$', r'$\sigma_\mathrm{sys}^{\mathrm{Tests}}$']

In [173]:
output = ' & ' + '& '.join(mean_labels+err_labels) + '& \\\\'
output += "\n \hline\hline"
fp_prec = [2,2,2,2,2,3,3,3,2,2,2,4,4]
for ii, parameter in enumerate(parameters):
    output += f"""\n {parameter}"""
    p10 = math.floor(np.log10(np.abs(mean['combined'][ii])))
    for jj, column in enumerate(mean_cols):
        if p10 in [-1,1]: p10=0
        output += f""" \n & ${{{format(round(mean[column][ii]/10**p10, fp_prec[ii]), f".{fp_prec[ii]:.0f}f")}}}"""
        output += f""" \pm {{{format(round(err[column][ii]/10**p10, fp_prec[ii]), f".{fp_prec[ii]:.0f}f")}}}"""
        if p10 == 0: output +=  "$"
        else: output +=  f"\\times 10^{{{p10}}}$"
    for jj, column in enumerate(err_cols):
        output += f""" \n & $\pm {{{format(round(err[column][ii]/10**p10, fp_prec[ii]), f".{fp_prec[ii]:.0f}f")}}}"""
        if p10 == 0: output +=  "$"
        else: output +=  f"\\times 10^{{{p10}}}$"   
    output += f" & ({sys_labels[sys_dominant[ii]]})"
    output += "\\\\"
    if ii in [1,4,7,10]:
        output += f"\n \hline"
output += "\hline\hline \n"

print(output)

 & $\mu^\mathrm{North}$& $\mu^\mathrm{South}$& $\mu$& $\sigma_\mathrm{sys}^{\mathrm{N}/\mathrm{S}}$& $\sigma_\mathrm{sys}^{\mathrm{Tests}}$& \\
 \hline\hline
 $\log_{10}\left(\rho^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$ 
 & ${-1.61} \pm {0.00}$ 
 & ${-1.52} \pm {0.00}$ 
 & ${-1.56} \pm {0.00}$ 
 & $\pm {0.05}$ 
 & $\pm {0.04}$ & ($z_\odot$)\\
 $\log_{10}\left(M^*_\mathrm{Halo}\,\left(\mathrm{M}_\odot\right)\right)$ 
 & ${18.60} \pm {0.05}$ 
 & ${19.31} \pm {0.06}$ 
 & ${18.95} \pm {0.05}$ 
 & $\pm {0.35}$ 
 & $\pm {0.44}$ & ($R$)\\
 \hline
 $\log_{10}\left(\rho_\mathrm{Tn} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$ 
 & ${-1.67} \pm {0.00}$ 
 & ${-1.57} \pm {0.00}$ 
 & ${-1.62} \pm {0.00}$ 
 & $\pm {0.05}$ 
 & $\pm {0.06}$ & ($z_\odot$)\\
 $\log_{10}\left(\rho_\mathrm{Tk} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$ 
 & ${-2.48} \pm {0.01}$ 
 & ${-2.41} \pm {0.01}$ 
 & ${-2.44} \pm {0.01}$ 
 & $\pm {0.03}$ 
 & $\pm {0.16}$ & ($A_v$)\

### RMS of systematics

In [121]:
columns = [r'$\mu^\mathrm{North}$', r'$\sigma^\mathrm{North}$', 
           r'$\mu^\mathrm{South}$', r'$\sigma^\mathrm{South}$',
           r'$\mu$', r'$\sigma$']
mean_cols = ['north', 'south', 'combined']
mean_labels = [r'$\mu^\mathrm{North}$', r'$\mu^\mathrm{South}$', r'$\mu$']

err_cols = ['sys_ns', 'sys_rms']
err_labels = [r'$\sigma_\mathrm{sys}^{\mathrm{N}/\mathrm{S}}$', r'$\sigma_\mathrm{sys}^{\mathrm{Tests}}$']

In [254]:
err['sys_max'] = np.max(err['sys_tests'], axis=0)

In [211]:
ii=12
contribution = err['sys_tests'][:,ii]/err['sys_max'][ii]
np.argwhere(contribution>0.7)[:,0][np.argsort(contribution[contribution>0.7])[::-1]]

array([4])

In [195]:
err['sys_tests'][:,ii]#/err['sys_max'][ii]

array([0.11917805, 0.04380571, 0.16075379, 0.15990228, 0.1194681 ,
       0.00536122, 0.13248849])

In [214]:
output = ' & ' + '& '.join(mean_labels+err_labels) + '& \\\\'
output += "\n \hline\hline"
fp_prec = [2,2,2,2,2,3,3,3,2,2,2,4,4]
for ii, parameter in enumerate(parameters):
    output += f"""\n {parameter}"""
    p10 = math.floor(np.log10(np.abs(mean['combined'][ii])))
    for jj, column in enumerate(mean_cols):
        if p10 in [-1,1]: p10=0
        output += f""" \n & ${{{format(round(mean[column][ii]/10**p10, fp_prec[ii]), f".{fp_prec[ii]:.0f}f")}}}"""
        output += f""" \pm {{{format(round(err[column][ii]/10**p10, fp_prec[ii]), f".{fp_prec[ii]:.0f}f")}}}"""
        if p10 == 0: output +=  "$"
        else: output +=  f"\\times 10^{{{p10}}}$"
    for jj, column in enumerate(err_cols):
        output += f""" \n & $\pm {{{format(round(err[column][ii]/10**p10, fp_prec[ii]), f".{fp_prec[ii]:.0f}f")}}}"""
        if p10 == 0: output +=  "$"
        else: output +=  f"\\times 10^{{{p10}}}$"   
    contribution = err['sys_tests'][:,ii]/err['sys_max'][ii]
    significant_contributors = np.argwhere(contribution>0.5)[:,0][np.argsort(contribution[contribution>0.5])[::-1]]
    print([systematics[ii] for ii in significant_contributors])
    output += f" & ({','.join([sys_labels[systematics[ii]] for ii in significant_contributors])})"
    output += "\\\\"
    if ii in [1,4,7,10]:
        output += f"\n \hline"
output += "\hline\hline \n"

print(output)

['z0', 'R']
['R', 'dust']
['z0']
['dust', 'gerr', 'R', 'poff', 'good']
['R']
['gerr', 'dust', 'R', 'good', 'poff']
['R', 'dust', 'gerr', 'poff', 'good']
['R', 'dust']
['mto', 'gerr']
['poff', 'z0']
['poff']
['poff']
['poff']
 & $\mu^\mathrm{North}$& $\mu^\mathrm{South}$& $\mu$& $\sigma_\mathrm{sys}^{\mathrm{N}/\mathrm{S}}$& $\sigma_\mathrm{sys}^{\mathrm{Tests}}$& \\
 \hline\hline
 $\log_{10}\left(\rho^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$ 
 & ${-1.61} \pm {0.00}$ 
 & ${-1.52} \pm {0.00}$ 
 & ${-1.56} \pm {0.00}$ 
 & $\pm {0.05}$ 
 & $\pm {0.06}$ & ($z_\odot$,$R$)\\
 $\log_{10}\left(M^*_\mathrm{Halo}\,\left(\mathrm{M}_\odot\right)\right)$ 
 & ${18.60} \pm {0.05}$ 
 & ${19.31} \pm {0.06}$ 
 & ${18.95} \pm {0.05}$ 
 & $\pm {0.35}$ 
 & $\pm {0.60}$ & ($R$,$A_v$)\\
 \hline
 $\log_{10}\left(\rho_\mathrm{Tn} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$ 
 & ${-1.67} \pm {0.00}$ 
 & ${-1.57} \pm {0.00}$ 
 & ${-1.62} \pm {0.00}$ 
 & $\pm {0.05}$ 
 & $\

# one-by-one convergence

In [79]:
err['sys_tests'] =  err['sys_tests_1b1'] * mean['combined']/sys_true_t['good']
err['sys_rms'] = np.sqrt(np.sum(err['sys_tests_1b1']**2, axis=0)) * mean['combined']/sys_true_t['good']
err['sys_max'] = np.max(err['sys_tests_1b1'], axis=0) * mean['combined']/sys_true_t['good']

In [80]:
columns = [r'$\mu^\mathrm{North}$', r'$\sigma^\mathrm{North}$', 
           r'$\mu^\mathrm{South}$', r'$\sigma^\mathrm{South}$',
           r'$\mu$', r'$\sigma$']
mean_cols = ['north', 'south', 'combined']
mean_labels = [r'$\mu^\mathrm{North}$', r'$\mu^\mathrm{South}$', r'$\mu$']

err_cols = ['sys_ns', 'sys_max']
err_labels = [r'$\sigma_\mathrm{sys}^{\mathrm{N}/\mathrm{S}}$', r'$\sigma_\mathrm{sys}^{\mathrm{Tests}}$']

In [81]:
ii=11
contribution = err['sys_tests'][:,ii]/err['sys_max'][ii]
np.argwhere(contribution>0.707)[:,0][np.argsort(contribution[contribution>0.7])[::-1]]

err['sys_tests'][:,ii]#/err['sys_max'][ii]

array([0.01619771, 0.00405768, 0.02183538, 0.02379656, 0.01454954,
       0.00559524, 0.0191318 ])

In [82]:
1/np.sqrt(2)

0.7071067811865475

In [83]:
param_short = ['rho', 'sd', 'Mhalo', 
               'rho_th', 'rho_tk', 'rho_H', 
               'tk/tn', 'H/tk',
               'sd_th', 'sd_tk', 'sd_H', 
               'h_th', 'h_tk', 'n_H', 
               'fG_th', 'fG_tk', 'fG_H', 'a1','a2']

In [103]:
err

{'north': array([2.45689229e-04, 7.72460719e-02, 2.74579721e+07, 2.18872437e-04,
        1.11426241e-04, 2.76835842e-07, 4.87159550e-03, 1.47870511e-05,
        1.23150890e-01, 1.11157717e-01, 3.73476591e-02, 2.50025517e-03,
        6.65993191e-03, 2.28382482e-02, 4.31849890e-04, 1.60517604e-04,
        9.54194431e-05, 4.68492878e-04, 2.26228221e-03]),
 'south': array([2.96975835e-04, 8.44471119e-02, 9.70098554e+06, 2.68429711e-04,
        9.67656973e-05, 1.80726032e-07, 3.33991091e-03, 7.67427686e-06,
        1.11868694e-01, 9.35391462e-02, 1.25551379e-02, 1.93783437e-03,
        4.77487145e-03, 1.99187489e-02, 4.24476948e-04, 1.53382172e-04,
        8.96344066e-05, 4.29079192e-04, 2.21038403e-03]),
 'combined': array([2.45689229e-04, 7.72460719e-02, 2.74579721e+07, 2.18872437e-04,
        1.11426241e-04, 2.76835842e-07, 4.87159550e-03, 1.47870511e-05,
        1.23150890e-01, 1.11157717e-01, 3.73476591e-02, 2.50025517e-03,
        6.65993191e-03, 2.28382482e-02, 4.31849890e-04, 1.6051

In [84]:
output = ' & ' + '& '.join(mean_labels+err_labels) + '& \\\\'
output += "\n \hline\hline"
fp_prec = [2,2,2, 2,2,2, 3,2, 2,2,2, 3,3,3, 2,2,2, 4,4]
for ii, parameter in enumerate(parameters):
    output += f"""\n {parameter}"""
    p10 = math.floor(np.log10(np.abs(mean['combined'][ii])))
    for jj, column in enumerate(mean_cols):
        if p10 in [-1,1]: p10=0
        output += f""" \n & ${{{format(round(mean[column][ii]/10**p10, fp_prec[ii]), f".{fp_prec[ii]:.0f}f")}}}"""
        output += f""" \pm {{{format(round(err[column][ii]/10**p10, fp_prec[ii]), f".{fp_prec[ii]:.0f}f")}}}"""
        if p10 == 0: output +=  "$"
        else: output +=  f"\\times 10^{{{p10}}}$"
    for jj, column in enumerate(err_cols):
        output += f""" \n & $\pm {{{format(round(err[column][ii]/10**p10, fp_prec[ii]), f".{fp_prec[ii]:.0f}f")}}}"""
        if p10 == 0: output +=  "$"
        else: output +=  f"\\times 10^{{{p10}}}$"   
    contribution = err['sys_tests'][:,ii]/err['sys_max'][ii]
    significant_contributors = np.argwhere(contribution>0.9)[:,0][np.argsort(contribution[contribution>0.9])[::-1]]
    if err['sys_max'][ii]/err['combined'][ii]>1e-1:
        print(param_short[ii].ljust(10), err['sys_rms'][ii], [systematics[ii] for ii in significant_contributors])
        output += f" & ({','.join([sys_labels[systematics[ii]] for ii in significant_contributors])})"
    else: 
        print(param_short[ii].ljust(10), 'no systematic')
        output += f" &"
    output += "\\\\"
    if ii in [2,5,7,10,13,16]:#[1,4,7,10]:
        output += f"\n \hline"
output += "\hline\hline \n"

print(output)

rho        0.0045924227897445715 ['z0']
sd         3.6653428258626874 ['R']
Mhalo      0.18201070617628795 ['R']
rho_th     0.0048554818672962435 ['z0']
rho_tk     0.0039393640134686 ['dust', 'gerr']
rho_H      2.3882356263043535e-06 ['R']
tk/tn      0.1391497482928282 ['dust', 'gerr']
H/tk       0.00015711728238451485 ['R']
sd_th      4.203016774421801 ['z0']
sd_tk      2.9996236412649298 ['gerr', 'dust']
sd_H       0.44981980968840785 ['R']
h_th       0.043942146879841215 ['gerr', 'dust']
h_tk       0.20809784481304608 ['R']
n_H        0.2796300192211623 ['R']
fG_th      0.013289231016685029 ['mto', 'gerr']
fG_tk      0.0019883319947642075 ['poff', 'dust']
fG_H       0.0006712421864395999 ['poff']
a1         0.006118384133843105 ['poff']
a2         0.029135751678264113 ['poff']
 & $\mu^\mathrm{North}$& $\mu^\mathrm{South}$& $\mu$& $\sigma_\mathrm{sys}^{\mathrm{N}/\mathrm{S}}$& $\sigma_\mathrm{sys}^{\mathrm{Tests}}$& \\
 \hline\hline
 $\rho^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\

rho        0.003471985251499379 ['z0']
sd         2.983677525654275 ['R']
Mhalo      no systematic
rho_th     0.003636381959657308 ['z0']
rho_tk     0.0031051497106129775 ['dust', 'gerr']
rho_H      1.9809331527729196e-06 ['R']
tk/tn      0.14645415544551002 ['dust', 'gerr']
H/tk       0.00017401210057457883 ['R']
sd_th      3.147735848989888 ['z0']
sd_tk      2.3644123612112335 ['gerr', 'dust']
sd_H       0.3731050648571948 ['R']
h_th       0.04394214690814578 ['gerr', 'dust']
h_tk       0.20809784475922946 ['R']
n_H        0.27963001747832145 ['R']
fG_th      0.013289230996769546 ['mto', 'gerr']
fG_tk      0.0019883319916043397 ['poff', 'dust']
fG_H       0.0006712421871095221 ['poff']
a1         0.006118384111784409 ['poff']
a2         0.029135751680895394 ['poff']
 & $\mu^\mathrm{North}$& $\mu^\mathrm{South}$& $\mu$& $\sigma_\mathrm{sys}^{\mathrm{N}/\mathrm{S}}$& $\sigma_\mathrm{sys}^{\mathrm{Tests}}$& \\
 \hline\hline
 $\rho^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3

/home/andy/py36/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars


rho        0.0034719852533002553 ['z0']
sd         2.9836775269729565 ['R']
Mhalo      13662999.25256136 ['mto']
rho_th     0.003636381948408342 ['z0']
rho_tk     0.003105149720333861 ['dust', 'gerr']
rho_H      1.9809331532229804e-06 ['R']
tk/tn      0.14645415530550515 ['dust', 'gerr']
H/tk       0.0001740121006024182 ['R']
sd_th      3.1477358458030986 ['z0']
sd_tk      2.3644123720986387 ['gerr', 'dust']
sd_H       0.373105066229296 ['R']
h_th       0.04394214691188767 ['gerr', 'dust']
h_tk       0.20809784504470685 ['R']
n_H        0.27963001760609213 ['R']
fG_th      0.013289231009887048 ['mto', 'gerr']
fG_tk      0.0019883319929476883 ['poff', 'dust']
fG_H       0.0006712421881393501 ['poff']
a1         0.006118384112094775 ['poff']
a2         0.029135751687081005 ['poff']
 & $\mu^\mathrm{North}$& $\mu^\mathrm{South}$& $\mu$& $\sigma_\mathrm{sys}^{\mathrm{N}/\mathrm{S}}$& $\sigma_\mathrm{sys}^{\mathrm{Tests}}$& \\
 \hline\hline
 $\rho^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\

/home/andy/py36/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars


rho        0.057009973864040366 ['z0']
rho_th     0.0698179536165172 ['z0']
rho_tk     0.31513991341422865 ['dust', 'gerr']
rho_H      0.05361070439195904 ['R']
sd         0.0688376653427889 ['R']
sd_th      0.11856032754259829 ['z0']
sd_tk      0.18354296447078536 ['gerr', 'dust']
sd_H       0.2926978476444885 ['R']
h_th       0.050791883475586226 ['gerr', 'dust']
h_tk       0.2701406755450863 ['R']
n_H        0.29517607914141203 ['R']
fG_th      0.004578025490513574 ['mto', 'gerr']
fG_tk      0.0019360174971795802 ['poff', 'dust']
fG_H       0.00044456705668470087 ['poff']
a1         0.006124245860450845 ['poff']
a2         0.0265323967626635 ['poff']
 & $\mu^\mathrm{North}$& $\mu^\mathrm{South}$& $\mu$& $\sigma_\mathrm{sys}^{\mathrm{N}/\mathrm{S}}$& $\sigma_\mathrm{sys}^{\mathrm{Tests}}$& \\
 \hline\hline
 $\log_{10}\left(\rho^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$ 
 & ${-1.607} \pm {0.004}$ 
 & ${-1.515} \pm {0.004}$ 
 & ${-1.561} \pm {0.004}$ 
 & $

rho        0.0570099738622254 ['z0']
rho_th     0.06981795351241203 ['z0']
rho_tk     0.3151399134675747 ['dust', 'gerr']
rho_H      0.05361070439098288 ['R']
h_th       0.05079188347447144 ['gerr', 'dust']
h_tk       0.27014067554512466 ['R']
n_H        0.29517607914420074 ['R']
fG_th      0.004578025495393188 ['mto', 'gerr']
fG_tk      0.0019360174971416945 ['poff', 'dust']
fG_H       0.0004445670567977436 ['poff']
a1         0.006124245863188897 ['poff']
a2         0.026532396762439913 ['poff']
 & $\mu^\mathrm{North}$& $\mu^\mathrm{South}$& $\mu$& $\sigma_\mathrm{sys}^{\mathrm{N}/\mathrm{S}}$& $\sigma_\mathrm{sys}^{\mathrm{Tests}}$& \\
 \hline\hline
 $\log_{10}\left(\rho^*_\mathrm{local} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$ 
 & ${-1.607} \pm {0.004}$ 
 & ${-1.515} \pm {0.004}$ 
 & ${-1.561} \pm {0.004}$ 
 & $\pm {0.046}$ 
 & $\pm {0.057}$ & ($z_\odot$)\\
 \hline
 $\log_{10}\left(\rho_\mathrm{Tn} \, \left(\mathrm{M}_\odot/\mathrm{pc}^3\right)\right)$ 
 & ${-1.670} \